## Image Processing Using Keras and Tensorflow - Chris Kim

This notebook was adapted from github user [nirmalyaghosh](https://github.com/nirmalyaghosh/deep-learning-vm), and user [fchollet](https://github.com/fchollet/keras/issues/3109). Lastly, most information was adapted from a great website called [Machine Learning Mastery](http://machinelearningmastery.com/).

The code below is my first attempt to use image processing using the tools made available in Python, namely `keras` and `TensorFlow` as the backend. This exploration was inspired by [doyleax](https://doyleax.github.io/Portfolio/capstone.html) and her Capstone project. Side profile images of 1780 pairs of shoes are transformed to be evaluated in a neural network to try to predict whether a shoe is classified as rare or not (determined by a metric composed of retail price and average deadstock price).

All images are taken from [StockX](www.stockx.com).

In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

np.random.seed(7)
%matplotlib inline

Using TensorFlow backend.


In [2]:
with open('shoenumbers.pickle', 'rb') as handle:
    X = pickle.load(handle, encoding='bytes')
with open('targetvars.pickle', 'rb') as handle:
    y = pickle.load(handle, encoding='bytes')

print(np.array(X).shape)
print(np.array(y).shape)

In [4]:
baseline = y.mean()
print(baseline)

0.502247191011


In [5]:
# Change size of images so it's not so huge when we pass it through keras
import cv2
new_X = []
for row in range(len(X)):
    new_X.append(cv2.resize(X[row], (140,100)))
np.array(new_X).shape

(1780, 100, 140, 3)

In [6]:
# Prepare data for training and testing
X = np.array(new_X)
X = X.astype('float32')
X /= 255.0
y = np.array(y)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [30]:
# Model creation by machinelearningmastery
num_classes=2
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100,140,3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [31]:
# Compile the model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# print(model.summary())

In [26]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 1335 samples, validate on 445 samples
Epoch 1/25
1335/1335 [==============================] - 103s - loss: 7.8471 - acc: 0.5019 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 2/25
1335/1335 [==============================] - 111s - loss: 7.9564 - acc: 0.5064 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 3/25
1335/1335 [==============================] - 116s - loss: 7.9564 - acc: 0.5064 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 4/25
1335/1335 [==============================] - 109s - loss: 7.9564 - acc: 0.5064 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 5/25
1335/1335 [==============================] - 107s - loss: 7.9564 - acc: 0.5064 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 6/25
1335/1335 [==============================] - 110s - loss: 7.9564 - acc: 0.5064 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 7/25
1335/1335 [==============================] - 111s - loss: 7.9564 - acc: 0.5064 - val_loss: 8.5118 - val_acc: 0.4719
Epoch 8/25
1335/1335 [==============================] - 113s - l

In [27]:
y_pred = model.predict_classes(X_test)

445/445 [==============================] - 6s     


In [29]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

## Conclusion
Neural Networking did not increase the accuracy of my model of baseline: 50%. In fact, after observing more closely, the model predicted all test data as one category. There are several factors that could have caused this preference.
- No image transformations
- Not enough epochs
- Incorrect parameters/layers used
- Not enough data
- All of the above

I believe that it is most likely all of the above, but most importantly, not enough data. Even Kaggle's dogs vs cats competition contained 25,000 images of dogs and cats, about 14x more images than this dataset. That doesn't even include all the pixels in each picture nor all the image transformations that are possible. But this was definitely a huge leap into the fast-growing field of deep learning.